In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
print(tf.__version__)

2.4.1


In [3]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
digits = pd.read_csv('../data/digit_recognizer_train.csv')

In [5]:
digits.shape

(42000, 785)

In [6]:
digits.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [7]:
X = digits.iloc[:, 1:]
y = digits.iloc[:, 0]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

#### GridSearchCV

In [14]:
def build_model(n_hidden=3, n_neurons=32, learning_rate=3e-3, input_shape=784):
    model = Sequential()
    
    model.add(InputLayer(input_shape=input_shape))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(10, activation="softmax"))
    
    optimizer = tf.keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics = ['accuracy'])
    
    return model

In [15]:
keras_class = tf.keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [16]:
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

param_distribs = {
    "n_hidden": [2],
    "n_neurons": [8],
    "learning_rate": [0.01, 0.001]
}

search_cv = GridSearchCV(keras_class, param_distribs, cv=3, verbose=2)
search_cv.fit(X_train, y_train.values, epochs=10, validation_split=0.2,
                  callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] learning_rate=0.01, n_hidden=2, n_neurons=8 .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
560/560 [==============================] - 1s 1ms/step - loss: 1.7011 - accuracy: 0.4190 - val_loss: 0.8280 - val_accuracy: 0.7246
Epoch 2/10
560/560 [==============================] - 0s 870us/step - loss: 0.7486 - accuracy: 0.7489 - val_loss: 0.5825 - val_accuracy: 0.8234
Epoch 3/10
560/560 [==============================] - 0s 884us/step - loss: 0.5401 - accuracy: 0.8365 - val_loss: 0.4770 - val_accuracy: 0.8650
Epoch 4/10
560/560 [==============================] - 1s 924us/step - loss: 0.4329 - accuracy: 0.8720 - val_loss: 0.4267 - val_accuracy: 0.8797
Epoch 5/10
560/560 [==============================] - 0s 874us/step - loss: 0.3862 - accuracy: 0.8843 - val_loss: 0.3969 - val_accuracy: 0.8906
Epoch 6/10
560/560 [==============================] - 1s 992us/step - loss: 0.3426 - accuracy: 0.8961 - val_loss: 0.3798 - val_accuracy: 0.8978
Epoch 7/10
560/560 [==============================] - 0s 820us/step - loss: 0.3227 - accuracy: 0.9059 - val_loss: 0.3689 - val_accuracy: 0

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.1s remaining:    0.0s


560/560 [==============================] - 1s 993us/step - loss: 1.9072 - accuracy: 0.3278 - val_loss: 1.0237 - val_accuracy: 0.6812
Epoch 2/10
560/560 [==============================] - 0s 835us/step - loss: 0.9264 - accuracy: 0.7174 - val_loss: 0.7372 - val_accuracy: 0.7942
Epoch 3/10
560/560 [==============================] - 0s 862us/step - loss: 0.6859 - accuracy: 0.8024 - val_loss: 0.6282 - val_accuracy: 0.8223
Epoch 4/10
560/560 [==============================] - 0s 785us/step - loss: 0.5795 - accuracy: 0.8310 - val_loss: 0.5642 - val_accuracy: 0.8397
Epoch 5/10
560/560 [==============================] - 0s 857us/step - loss: 0.5063 - accuracy: 0.8539 - val_loss: 0.5191 - val_accuracy: 0.8571
Epoch 6/10
560/560 [==============================] - 0s 818us/step - loss: 0.4640 - accuracy: 0.8681 - val_loss: 0.4827 - val_accuracy: 0.8652
Epoch 7/10
560/560 [==============================] - 0s 840us/step - loss: 0.4436 - accuracy: 0.8720 - val_loss: 0.4552 - val_accuracy: 0.8717
Epo

[CV] ..... learning_rate=0.001, n_hidden=2, n_neurons=8, total=   6.4s
Epoch 1/10


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   36.8s finished


840/840 [==============================] - 1s 1ms/step - loss: 1.8260 - accuracy: 0.3733 - val_loss: 1.0052 - val_accuracy: 0.7326
Epoch 2/10
840/840 [==============================] - 1s 856us/step - loss: 0.8676 - accuracy: 0.7682 - val_loss: 0.6567 - val_accuracy: 0.8327
Epoch 3/10
840/840 [==============================] - 1s 927us/step - loss: 0.5815 - accuracy: 0.8450 - val_loss: 0.5331 - val_accuracy: 0.8571
Epoch 4/10
840/840 [==============================] - 1s 912us/step - loss: 0.4759 - accuracy: 0.8704 - val_loss: 0.4667 - val_accuracy: 0.8786
Epoch 5/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4082 - accuracy: 0.8897 - val_loss: 0.4176 - val_accuracy: 0.8891
Epoch 6/10
840/840 [==============================] - 1s 1ms/step - loss: 0.3511 - accuracy: 0.9025 - val_loss: 0.3931 - val_accuracy: 0.8929
Epoch 7/10
840/840 [==============================] - 1s 953us/step - loss: 0.3339 - accuracy: 0.9052 - val_loss: 0.3778 - val_accuracy: 0.8952
Epoch 8/1

GridSearchCV(cv=3,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff83d6b4040>,
             param_grid={'learning_rate': [0.01, 0.001], 'n_hidden': [2],
                         'n_neurons': [8]},
             verbose=2)

In [17]:
print("Best Parameters: ", search_cv.best_params_)
print("Best Scores: ", search_cv.best_score_)
print("Best Estimator: ", search_cv.best_estimator_)

Best Parameters:  {'learning_rate': 0.01, 'n_hidden': 2, 'n_neurons': 8}
Best Scores:  0.8900595108668009
Best Estimator:  <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff808456df0>


In [18]:
model = search_cv.best_estimator_

In [19]:
from sklearn.metrics import accuracy_score

In [38]:
model.predict(X_test)

array([8, 1, 9, ..., 3, 0, 9])

In [39]:
y_pred = model.predict(X_test)

In [40]:
y_pred

array([8, 1, 9, ..., 3, 0, 9])

In [41]:
accuracy_score(y_test, y_pred)

0.9060714285714285